In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/'My Drive'/'kaggle'/'LinkingWritingProcess'

/content/drive/My Drive/kaggle/LinkingWritingProcess


In [3]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.6 MB/s eta 0:00:00


In [4]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.6 MB/s eta 0:00:00


In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold

import warnings
warnings.filterwarnings('ignore')
import datetime as dt

from catboost import CatBoostRegressor
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor,GradientBoostingRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import optuna
from sklearn.ensemble import AdaBoostRegressor

from scipy.optimize import minimize

./data/sample_submission.csv
./data/test_logs.csv
./data/train_logs.csv
./data/train_scores.csv


## データの読み込み

In [6]:
# 訓練データ読み込み
train_df = pd.read_csv("./data/train_logs.csv")
train_scores = pd.read_csv("./data/train_scores.csv")
# 検証データ読み込み
test_df = pd.read_csv("./data/test_logs.csv")

print("train_df： ", train_df.shape)
print("train_scores: ", train_scores.shape)
print("test_df: ", test_df.shape)

train_df：  (8405898, 11)
train_scores:  (2471, 2)
test_df:  (6, 11)


In [7]:
# 訓練データの表示確認
train_df.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [8]:
# 検証データの表示確認
test_df.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0


## データ概要

In [9]:
# 各項目のデータ型の確認
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405898 entries, 0 to 8405897
Data columns (total 11 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   id               object
 1   event_id         int64 
 2   down_time        int64 
 3   up_time          int64 
 4   action_time      int64 
 5   activity         object
 6   down_event       object
 7   up_event         object
 8   text_change      object
 9   cursor_position  int64 
 10  word_count       int64 
dtypes: int64(6), object(5)
memory usage: 705.5+ MB


In [10]:
# データの統計データを確認
train_df.describe()

,event_id,down_time,up_time,action_time,cursor_position,word_count
count,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06
mean,2.067649e+03,7.935603e+05,7.936584e+05,9.808498e+01,1.222964e+03,2.314687e+02
std,1.588284e+03,5.149451e+05,5.149428e+05,2.533985e+02,9.485242e+02,1.759088e+02
min,1.000000e+00,1.060000e+02,2.520000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.520000e+02,3.731842e+05,3.732820e+05,6.600000e+01,4.990000e+02,9.600000e+01
50%,1.726000e+03,7.208860e+05,7.209800e+05,9.300000e+01,1.043000e+03,2.000000e+02
75%,2.926000e+03,1.163042e+06,1.163141e+06,1.220000e+02,1.706000e+03,3.270000e+02
max,1.287600e+04,8.313630e+06,8.313707e+06,4.474700e+05,7.802000e+03,1.326000e+03


In [11]:
# 欠損値の確認(訓練データ)
for col in train_df.columns:
    pct_missing = np.mean(train_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
event_id - 0%
down_time - 0%
up_time - 0%
action_time - 0%
activity - 0%
down_event - 0%
up_event - 0%
text_change - 0%
cursor_position - 0%
word_count - 0%


In [12]:
# 欠損値の確認(検証データ)
for col in test_df.columns:
    pct_missing = np.mean(test_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
event_id - 0%
down_time - 0%
up_time - 0%
action_time - 0%
activity - 0%
down_event - 0%
up_event - 0%
text_change - 0%
cursor_position - 0%
word_count - 0%


## 特徴量の作成
https://www.kaggle.com/code/polyakovstepan/random-forest-atboost-xgb-baseline

In [13]:
def summary_time(df):
    result = df.groupby('id')['action_time'].sum().reset_index()
    result.rename(columns={'action_time': 'summary_time'}, inplace=True)
    return result

def start_pause(df):
    result = df.groupby('id')['down_time'].min().reset_index()
    result.rename(columns={'down_time':'start_pause'}, inplace=True)
    return result

def enter_click(df):
    copy_df = df
    copy_df['enter_click'] = (copy_df['down_event'] == 'Enter')
    copy_df = copy_df.groupby('id')['enter_click'].sum().reset_index()
    return copy_df

def space_click(df):
    copy_df = df
    copy_df['space_click'] = (copy_df['down_event'] == 'Space')
    copy_df = copy_df.groupby('id')['space_click'].sum().reset_index()
    return copy_df

def backspace_click(df):
    copy_df = df
    copy_df['backspace_click'] = (copy_df['down_event'] == 'Backspace')
    copy_df = copy_df.groupby('id')['backspace_click'].sum().reset_index()
    return copy_df

def symbol_length(df):
    result = df.groupby('id')['cursor_position'].max().reset_index()
    result.rename(columns={'cursor_position':'symbol_length'}, inplace=True)
    return result

def text_length(df):
    result = df.groupby('id')['word_count'].max().reset_index()
    return result

def nonproduction_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Nonproduction').mean() * 100).reset_index()
    result.rename(columns={'activity': 'nonproduction_feature'}, inplace=True)
    return result

def input_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Input').mean() * 100).reset_index()
    result.rename(columns={'activity': 'input_feature'}, inplace=True)
    return result

def remove_feature(df):
    result = df.groupby('id')['activity'].apply(lambda x: (x == 'Remove/Cut').mean() * 100).reset_index()
    result.rename(columns={'activity': 'remove_feature'}, inplace=True)
    return result

def mean_action_time(df):
    result = df.groupby('id')['action_time'].mean().reset_index()
    result.rename(columns={'action_time':'mean_action_time'}, inplace=True)
    return result

def replace_feature(df):
    result = df[df['activity'] == 'Replace'].groupby('id').size().reset_index(name='replace_feature')
    return result

def text_change_unique(df):
    result = df.groupby('id')['text_change'].nunique().reset_index()
    result.rename(columns={'text_change': 'tch_unique'}, inplace=True)
    return result

def sentence_size_feature(df):
    result = df[(df['text_change'] == '.') & (df['down_event'] != 'Backspace')].groupby('id').size().reset_index(name = 'number_sentence')
    return result

In [14]:
def getDataset(train_df):
    new_df = summary_time(train_df)

    functions = [
        start_pause,enter_click,space_click,backspace_click,symbol_length,text_length,
        nonproduction_feature, input_feature, remove_feature, mean_action_time, replace_feature,
        text_change_unique, sentence_size_feature
    ]

    for func in functions:
        result_df = func(train_df)
        new_df = pd.merge(new_df, result_df, on='id', how='outer')

    return new_df

In [15]:
# 訓練データの特徴量生成
df = getDataset(train_df)
print('train_info: ', df.shape)
df.head()

train_info:  (2471, 15)


,id,summary_time,start_pause,enter_click,space_click,backspace_click,symbol_length,word_count,nonproduction_feature,input_feature,remove_feature,mean_action_time,replace_feature,tch_unique,number_sentence
0,001519c8,297243,4526,4,357,417,1539,256,4.693000,78.607743,16.308174,116.246774,7.0,17,21.0
1,0022f953,275391,30623,6,391,260,1676,323,10.350448,78.973105,10.594947,112.221271,1.0,12,15.0
2,0042269b,421201,4441,17,552,439,2291,404,4.231141,84.985493,10.614120,101.837766,7.0,19,21.0
3,0059420b,189596,41395,3,243,152,1047,206,6.362468,83.804627,9.704370,121.848329,1.0,10,13.0
4,0075873a,313702,78470,10,324,517,1402,252,2.844725,76.728566,20.426709,123.943896,NaN,9,23.0


In [16]:
# 検証データの特徴量生成
test = getDataset(test_df)
print('test_info: ', test.shape)
test.shape

test_info:  (3, 15)


(3, 15)

## 欠損値消去

In [17]:
# 訓練データの欠損値確認
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
summary_time - 0%
start_pause - 0%
enter_click - 0%
space_click - 0%
backspace_click - 0%
symbol_length - 0%
word_count - 0%
nonproduction_feature - 0%
input_feature - 0%
remove_feature - 0%
mean_action_time - 0%
replace_feature - 56%
tch_unique - 0%
number_sentence - 0%


In [18]:
# 検証データの欠損値確認
for col in test.columns:
    pct_missing = np.mean(test[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

id - 0%
summary_time - 0%
start_pause - 0%
enter_click - 0%
space_click - 0%
backspace_click - 0%
symbol_length - 0%
word_count - 0%
nonproduction_feature - 0%
input_feature - 0%
remove_feature - 0%
mean_action_time - 0%
replace_feature - 100%
tch_unique - 0%
number_sentence - 100%


In [19]:
# replace_featureとnumber_sentenceに欠損値が確認される
# この２つの特徴量を消去する
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

for col in  numeric_cols:
    missing = df[col].isnull()
    num_missing = np.sum(missing)

    if num_missing > 0:
        print('imputing missing values for: {}'.format(col))
        df['{}_ismissing'.format(col)] = missing
        med = test[col].median()
        df[col] = df[col].fillna(med)

imputing missing values for: replace_feature
imputing missing values for: number_sentence


In [20]:
df_numeric = test.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
for col in numeric_cols:
    missing = test[col].isnull()
    num_missing = np.sum(missing)

    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print('imputing missing values for: {}'.format(col))
        test['{}_ismissing'.format(col)] = missing
        med = test[col].median()
        test[col] = test[col].fillna(med)

imputing missing values for: replace_feature
imputing missing values for: number_sentence


In [21]:
print('train_info: ', df.shape, 'test_info: ', test.shape)

train_info:  (2471, 17) test_info:  (3, 17)


In [22]:
num_rows = len(df.index)
low_information_cols = []

for col in df.columns:
  cnts = df[col].value_counts(dropna=False)
  top_pct = (cnts/num_rows).iloc[0]

  if top_pct > 0.95:
    low_information_cols.append(col)
    print(' {0}: {1:.5f}%'.format(col, top_pct*100))
    print(cnts)
    print()

 number_sentence_ismissing: 99.95953%
False    2470
True        1
Name: number_sentence_ismissing, dtype: int64



In [23]:
df = df.drop(columns = ['number_sentence_ismissing'], axis=1)
test = test.drop(columns = ['number_sentence_ismissing'], axis=1)

print('train_info: ', df.shape, 'test_info: ', test.shape)

train_info:  (2471, 16) test_info:  (3, 16)


In [24]:
df = pd.merge(df, train_scores, on='id', how='outer')
print('train_info: ', df.shape, 'test_info: ', test.shape)

train_info:  (2471, 17) test_info:  (3, 16)


In [25]:
# 欠損値に平均をはめる
df['replace_feature'].fillna(df['replace_feature'].mean(), inplace=True)
test['replace_feature'].fillna(test['replace_feature'].mean(), inplace=True)
df['number_sentence'].fillna(df['number_sentence'].mean(), inplace=True)
test['number_sentence'].fillna(test['number_sentence'].mean(), inplace=True)

## Baseline

In [26]:
# 説明変数と目的変数を設定
X = df.drop(columns = ['id', 'score'], axis=1)
y = df.score
X_total = test.drop(columns=['id'], axis=1)

print('train = ', X.shape, 'test = ', X_total.shape, 'target = ', y.shape)

train =  (2471, 15) test =  (3, 15) target =  (2471,)


In [27]:
# モデルの定義
model_1 = CatBoostRegressor(verbose=0)
model_2 = xgb.XGBRegressor()
model_3 = lgb.LGBMRegressor()
model_4 = RandomForestRegressor()
model_5 = HistGradientBoostingRegressor()

# クロスバリデーションを行なって精度の比較
print('Score baseline CatBoostRegressor: ', cross_val_score(model_1, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline XGBRegressor: ', cross_val_score(model_2, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline LGBMRegressor: ', cross_val_score(model_3, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline RandomForestRegressor: ', cross_val_score(model_4, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())
print('Score baseline HostGradientBoostingRegressor: ', cross_val_score(model_5, X, y, cv=5, scoring='neg_root_mean_squared_error').mean())

Score baseline CatBoostRegressor:  -0.6889205925773207
Score baseline XGBRegressor:  -0.7256322929966512
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000462 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2713
[LightGBM] [Info] Number of data points in the train set: 1976, number of used features: 15
[LightGBM] [Info] Start training from score 3.710273
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000533 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2711
[LightGBM] [Info] Number of data points in the train set: 1977, number of used features: 15
[LightGBM] [Info] Start training from score 3.709914
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000326 seconds.
You can set `force_col_wise=true` to remove t

## データ準備

In [28]:
def variance_threshold(df, th):
  var_thres = VarianceThreshold(threshold=th)
  var_thres.fit(df)
  new_cols = var_thres.get_support()
  return df.iloc[:, new_cols]

In [29]:
X_fit = variance_threshold(X, 0.2)
list_name = (X_fit.columns)
X_total_fit = X_total[list_name]

print('Shape test = ', X_total_fit.shape)
print('Shape train = ', X_fit.shape)

Shape test =  (3, 15)
Shape train =  (2471, 15)


In [30]:
X_fit = X_fit.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
X_total_fit = X_total_fit.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

print('Shape train = ', X_fit.shape)
print('Shape test = ', X_total_fit.shape)

Shape train =  (2471, 15)
Shape test =  (3, 15)


In [31]:
# 訓練データの標準化
std_ = StandardScaler()
X_std = std_.fit_transform(X_fit)
X_total_std = std_.fit_transform(X_total_fit)

## Split and Train

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.3, random_state=42)

In [33]:
def objective(trial):

  ### define params grid to search maximum accuracy ###
  n_estimators = trial.suggest_int('n_estimators', 50, 120)
  max_depth = trial.suggest_int('max_depth', 10, 16)
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 15, 25)
  criterion = trial.suggest_categorical('criterion', ['friedman_mse', 'absolute_error'])

  ### modeling with suggested params ###
  model = ExtraTreesRegressor(n_estimators = n_estimators,
                                 max_depth = max_depth,
                                 max_leaf_nodes = max_leaf_nodes,
                                 criterion = criterion,
                                 random_state = 0)

  ### fit ###
  model.fit(X_train, y_train)
  accuracy = mean_squared_error(y_test, model.predict(X_test))
  return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2)
study.best_value
study.best_trial.params

[I 2023-11-03 08:00:56,818] A new study created in memory with name: no-name-3992b8a1-bff2-46f2-8ae3-b15d13502366
[I 2023-11-03 08:01:08,227] Trial 0 finished with value: 0.46739259602425876 and parameters: {'n_estimators': 100, 'max_depth': 11, 'max_leaf_nodes': 24, 'criterion': 'absolute_error'}. Best is trial 0 with value: 0.46739259602425876.
[I 2023-11-03 08:01:11,955] Trial 1 finished with value: 0.47227654068812186 and parameters: {'n_estimators': 72, 'max_depth': 16, 'max_leaf_nodes': 21, 'criterion': 'absolute_error'}. Best is trial 1 with value: 0.47227654068812186.


{'n_estimators': 72,
 'max_depth': 16,
 'max_leaf_nodes': 21,
 'criterion': 'absolute_error'}

In [34]:
def objective(trial):
  params = {
      "iterations": 1000,
      "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
      "depth": trial.suggest_int("depth", 1, 10),
      "subsample": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
      "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
      "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
  }

  model = CatBoostRegressor(**params, silent=True)
  model.fit(X_train, y_train)
  predictions = model.predict(X_test)
  rmse = mean_squared_error(y_test, predictions, squared=False)
  return rmse

study_cat = optuna.create_study(direction='minimize')
study_cat.optimize(objective, n_trials=3)

print('Best hyperparameters: ', study_cat.best_params)
print('Best RMSE: ', study_cat.best_value)

[I 2023-11-03 08:01:11,973] A new study created in memory with name: no-name-17c4c9a4-26ef-45e1-821e-9461b5c01c01
[I 2023-11-03 08:01:40,252] Trial 0 finished with value: 0.6698631316481025 and parameters: {'learning_rate': 0.018424143042471076, 'depth': 10, 'colsample_bylevel': 0.5585474465157528, 'min_data_in_leaf': 77}. Best is trial 0 with value: 0.6698631316481025.
[I 2023-11-03 08:01:41,628] Trial 1 finished with value: 0.6681646285444782 and parameters: {'learning_rate': 0.06318006446966139, 'depth': 5, 'colsample_bylevel': 0.37154007915826204, 'min_data_in_leaf': 40}. Best is trial 1 with value: 0.6681646285444782.
[I 2023-11-03 08:01:43,346] Trial 2 finished with value: 0.6936596151723267 and parameters: {'learning_rate': 0.0023057951137501888, 'depth': 7, 'colsample_bylevel': 0.2046604934207481, 'min_data_in_leaf': 74}. Best is trial 1 with value: 0.6681646285444782.


Best hyperparameters:  {'learning_rate': 0.06318006446966139, 'depth': 5, 'colsample_bylevel': 0.37154007915826204, 'min_data_in_leaf': 40}
Best RMSE:  0.6681646285444782


In [35]:
# ランダムフォレストの最適パラメータを探索
def objective(trial):
  criterion = trial.suggest_categorical('criterion', ['friedman_mse', 'squared_error'])
  max_depth = trial.suggest_int('max_depth', 1, 1000)
  max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 1, 1000)
  n_estimators = trial.suggest_int('n_estimators', 1, 1000)
  min_samples_split = trial.suggest_int('min_samples_split', 2, 5)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

  regr = RandomForestRegressor(criterion=criterion,
                                 max_depth=max_depth, max_features=max_features,
                                 max_leaf_nodes=max_leaf_nodes, n_estimators=n_estimators,
                                 min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                 n_jobs=2)
  regr.fit(X_train, y_train)
  predictions = regr.predict(X_test)
  rmse = mean_squared_error(y_test, predictions, squared=False)

  return rmse

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective, n_trials=1)

print('Best hyperparameters: ', study_rf.best_params)
print('Best RMSE: ', study_rf.best_value)

[I 2023-11-03 08:01:43,367] A new study created in memory with name: no-name-682a312e-b4a3-48e2-a05d-35c5d77a6e70
[I 2023-11-03 08:01:49,051] Trial 0 finished with value: 0.6654545648163044 and parameters: {'criterion': 'squared_error', 'max_depth': 341, 'max_features': 'auto', 'max_leaf_nodes': 55, 'n_estimators': 480, 'min_samples_split': 5, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.6654545648163044.


Best hyperparameters:  {'criterion': 'squared_error', 'max_depth': 341, 'max_features': 'auto', 'max_leaf_nodes': 55, 'n_estimators': 480, 'min_samples_split': 5, 'min_samples_leaf': 6}
Best RMSE:  0.6654545648163044


In [36]:
# LightBGM回帰の最適パラメータを探索
def objective(trial):
  param = {
      'metric': 'rmse',
      'random_state': 48,
      'n_estimators': 20000,
      'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
      'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
      'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
      'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
      'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
      'max_depth': trial.suggest_categorical('max_depth', [10, 20, 100]),
      'num_leaves': trial.suggest_int('num_leaves', 1, 1000),
      'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
      'cat_smooth': trial.suggest_int('min_data_per_groups', 1, 100)
  }
  model = lgb.LGBMRegressor(**param)
  #model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=False)
  model.fit(X_train,y_train,eval_set=[(X_test,y_test)],callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=False), lgb.log_evaluation(0)])
  preds = model.predict(X_test)

  rmse = mean_squared_error(y_test, preds, squared=False)
  return rmse

study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective, n_trials=5)
print('Number of finished trials: ', len(study_lgbm.trials))
print('Best trial: ', study_lgbm.best_trial.params)

[I 2023-11-03 08:01:49,078] A new study created in memory with name: no-name-c59b5e05-9951-4c06-897c-2bd3de5ee0f7


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000391 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2710
[LightGBM] [Info] Number of data points in the train set: 1729, number of used features: 15
[LightGBM] [Info] Start training from score 3.713997
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

[I 2023-11-03 08:01:49,539] Trial 0 finished with value: 0.6596659621131566 and parameters: {'reg_alpha': 0.004683047556746833, 'reg_lambda': 0.01185225482933502, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.02, 'max_depth': 10, 'num_leaves': 784, 'min_child_samples': 54, 'min_data_per_groups': 17}. Best is trial 0 with value: 0.6596659621131566.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000407 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2710
[LightGBM] [Info] Number of data points in the train set: 1729, number of used features: 15
[LightGBM] [Info] Start training from score 3.713997
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

[I 2023-11-03 08:01:50,418] Trial 1 finished with value: 0.667484587929959 and parameters: {'reg_alpha': 0.0013407391578663377, 'reg_lambda': 0.33020894323685407, 'colsample_bytree': 0.6, 'subsample': 0.8, 'learning_rate': 0.014, 'max_depth': 20, 'num_leaves': 13, 'min_child_samples': 143, 'min_data_per_groups': 90}. Best is trial 0 with value: 0.6596659621131566.


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000434 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2710
[LightGBM] [Info] Number of data points in the train set: 1729, number of used features: 15
[LightGBM] [Info] Start training from score 3.713997
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

[I 2023-11-03 08:01:53,170] Trial 3 finished with value: 0.6632103910714512 and parameters: {'reg_alpha': 0.7350889808189157, 'reg_lambda': 0.0026459683897414126, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.02, 'max_depth': 10, 'num_leaves': 58, 'min_child_samples': 27, 'min_data_per_groups': 27}. Best is trial 2 with value: 0.6589064160962841.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000433 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2710
[LightGBM] [Info] Number of data points in the train set: 1729, number of used features: 15
[LightGBM] [Info] Start training from score 3.713997
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

[I 2023-11-03 08:01:53,966] Trial 4 finished with value: 0.6712633999903421 and parameters: {'reg_alpha': 0.16978306802251514, 'reg_lambda': 0.0014156890672209071, 'colsample_bytree': 0.6, 'subsample': 0.4, 'learning_rate': 0.01, 'max_depth': 20, 'num_leaves': 40, 'min_child_samples': 225, 'min_data_per_groups': 11}. Best is trial 2 with value: 0.6589064160962841.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [37]:
def objective(trial):

    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1, 500),
        'learning_rate':trial.suggest_float("learning_rate", 1e-3, 0.5, log=True),
        'subsample': trial.suggest_float("subsample", 0.05, 1.0),
        'max_depth':trial.suggest_int("max_depth", 1, 10)
    }
    model = GradientBoostingRegressor(**param)

    model.fit(X_train,y_train)

    preds = model.predict(X_test)

    rmse = mean_squared_error(y_test, preds,squared=False)

    return rmse

study_gbr = optuna.create_study(direction='minimize')
study_gbr.optimize(objective, n_trials=5)
print('Number of finished trials:', len(study_gbr.trials))
print('Best trial:', study_gbr.best_trial.params)

[I 2023-11-03 08:01:54,007] A new study created in memory with name: no-name-0350e5e9-258c-4c8d-b702-f405cadb3a5f
[I 2023-11-03 08:02:03,927] Trial 0 finished with value: 0.6755557761948296 and parameters: {'n_estimators': 399, 'learning_rate': 0.07918613352231385, 'subsample': 0.6369439704648463, 'max_depth': 9}. Best is trial 0 with value: 0.6755557761948296.
[I 2023-11-03 08:02:06,324] Trial 1 finished with value: 0.6986891577904336 and parameters: {'n_estimators': 199, 'learning_rate': 0.007035935090411382, 'subsample': 0.6570744180941247, 'max_depth': 3}. Best is trial 0 with value: 0.6755557761948296.
[I 2023-11-03 08:02:10,581] Trial 2 finished with value: 0.8213016066818447 and parameters: {'n_estimators': 232, 'learning_rate': 0.0018803218643296232, 'subsample': 0.6210726052666725, 'max_depth': 8}. Best is trial 0 with value: 0.6755557761948296.
[I 2023-11-03 08:02:24,191] Trial 3 finished with value: 0.754348415220029 and parameters: {'n_estimators': 437, 'learning_rate': 0.0

Number of finished trials: 5
Best trial: {'n_estimators': 399, 'learning_rate': 0.07918613352231385, 'subsample': 0.6369439704648463, 'max_depth': 9}


In [38]:
r1 = ExtraTreesRegressor(**study.best_trial.params)
r2 = CatBoostRegressor(**study_cat.best_params, verbose = 0)
r3 = RandomForestRegressor(**study_rf.best_params)
r4 = lgb.LGBMRegressor(**study_lgbm.best_trial.params)
r5 = AdaBoostRegressor()
r6 = GradientBoostingRegressor(**study_gbr.best_trial.params)


r1.fit(X_train,y_train)
r2.fit(X_train,y_train)
r3.fit(X_train,y_train)
r4.fit(X_train,y_train)
r5.fit(X_train,y_train)
r6.fit(X_train,y_train)

[LightGBM] [Warning] Unknown parameter: min_data_per_groups
[LightGBM] [Warning] Unknown parameter: min_data_per_groups
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000465 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2710
[LightGBM] [Info] Number of data points in the train set: 1729, number of used features: 15
[LightGBM] [Info] Start training from score 3.713997
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

GradientBoostingRegressor(learning_rate=0.07918613352231385, max_depth=9,
                          n_estimators=399, subsample=0.6369439704648463)

## Best weights

In [39]:
# 各モデルのペストパラメータを見つける
def find_best_weight(preds, target):
  def _validate_func(weights):
    ''' scipy minimize will pass the weights as a numpy array '''
    final_prediction = 0
    for weight, prediction in zip(weights, preds):
      final_prediction += weight * prediction
    return mean_squared_error(target, final_prediction)

  starting_values = [1/len(preds)] * len(preds)
  cons = ({'type':'eq', 'fun': lambda w: 1-sum(w)})
  # our weights are bound between 0 and 1
  bounds = [(0, 1)] * len(preds)

  res = minimize(_validate_func, starting_values, method='SLSQP', bounds=bounds, constraints=cons)

  print('Ensemble Score: {best_score}'.format(best_score=(1-res['fun'])))
  print('Best Weights: {weights}'.format(weights=res['x']))

  return res

In [40]:
oof1 = r1.predict(X_test)
oof2 = r2.predict(X_test)
oof3 = r3.predict(X_test)
oof4 = r4.predict(X_test)
oof5 = r5.predict(X_test)
oof6 = r6.predict(X_test)

[LightGBM] [Warning] Unknown parameter: min_data_per_groups


In [41]:
res = find_best_weight([oof1, oof2, oof3, oof4, oof5, oof6], y_test)

Ensemble Score: 0.5755011028576433
Best Weights: [0.05612221 0.27584314 0.17448762 0.         0.22978789 0.26375915]


In [49]:
yy = '123456'
dd = {yy[i]:k for i,k in enumerate(res['x'])}
print(dd)

{'1': 0.05612220691372842, '2': 0.27584313904488567, '3': 0.1744876179445054, '4': 0.0, '5': 0.22978789057657517, '6': 0.2637591455203054}


## Submit

In [50]:
sample = pd.read_csv('./data/sample_submission.csv')

In [51]:
sample['score'] = r1.predict(X_total_std)*dd['1'] + \
                  r2.predict(X_total_std)*dd['2'] + \
                  r3.predict(X_total_std)*dd['3'] + \
                  r4.predict(X_total_std)*dd['4'] + \
                  r5.predict(X_total_std)*dd['5'] + \
                  r6.predict(X_total_std)*dd['6']
sample

[LightGBM] [Warning] Unknown parameter: min_data_per_groups


,id,score
0,0000aaaa,3.300995
1,2222bbbb,3.807938
2,4444cccc,4.138670


In [52]:
sample.to_csv('submission.csv', index=False)